In [1]:
import numpy as np
import pandas as pd
import igraph
import random as rnd
import csv

In [49]:
class NS:

    # 初始化
    def __init__(self, num_nodes, p_rewire, z, a, mod, end_ratio, echo):
        self.num_nodes = num_nodes
        self.p_rewire = p_rewire
        self.z = z
        self.a = a
        self.mod = mod
        self.end_ratio = end_ratio
        self.echo = echo

        # 1. create the network
        # create a small world without randomness
        g = igraph.Graph.Watts_Strogatz(dim = 1,
            size = self.num_nodes,
            nei = int(z/2),
            p = 0)
        # Randomly rewires the graph while preserving the degree distribution
        g.rewire(n = int(g.ecount() * 0.5 * self.p_rewire),
            mode = "simple")

        # 2. set up states
        # the initial node and its neighbors are activited
        focal_node = rnd.sample(range(self.num_nodes), 1)[0]
        initial_activate = g.neighborhood(vertices=focal_node,
            order = 1)
        self.State = np.zeros(shape = (self.num_nodes,))
        self.State[initial_activate, ] = 1


        # 3. weight matrix
        adj_matrix = g.get_adjacency_sparse() # get adjacency matrix
        # mode1
        if mod == 1:
            self.WM = adj_matrix[:,:]
        # mode2 and mode3
        else:
            # calculate edge betweeness
            Centrality = np.array(g.edge_betweenness())
            cut_points = np.percentile(Centrality, q=[25, 75])

            # change the value based on cut_points
            Centrality[Centrality < cut_points[0]] = 1  # ties embedded in small circle
            Centrality[(Centrality >= cut_points[0]) & (Centrality < cut_points[1])] = 2
            Centrality[Centrality >= cut_points[1]] = 3 # long-range ties
            # mode2
            if mod == 2:
                self.WM = adj_matrix[:,:]
                # get index of strong ties
                id_strong_mod2 = np.where(Centrality == 1)[0]
                # change the matrix
                for source_target in g.es[list(id_strong_mod2)]:
                    i, j = source_target.tuple
                    self.WM[i, j] = 5
                    self.WM[j, i] = 5
            # mode3
            else:
                self.WM = adj_matrix[:,:]
                # get index of strong ties
                id_strong_embed = np.random.choice(np.where(Centrality == 1)[0], 
                    int(g.ecount() * .2), replace=False)
                id_strong_long_range = np.random.choice(np.where(Centrality == 3)[0], 
                    int(g.ecount() * .05), replace=False)
                id_strong_mod3 = np.hstack((id_strong_embed, id_strong_long_range))
                # chang the matrix
                for source_target in g.es[list(id_strong_mod3)]:
                    i, j = source_target.tuple
                    self.WM[i, j] = 5
                    self.WM[j, i] = 5
    
    # 模拟函数
    def start_contagion(self):

        times = 0 # record steps
        prop_path = [self.State.mean()] # record proportions
        f = open("sim_record.csv", mode = 'a', newline='')
        csv_writer = csv.writer(f)

        # contagion process
        while prop_path[-1] <= self.end_ratio:

            # nonactivated nodes index:
            nonactivated_index = np.where(self.State != 1)[0]

            # update these nonactivated nodes in random order
            np.random.shuffle(nonactivated_index)
            for i in nonactivated_index:
                total_effects = np.dot(self.State, self.WM[:, i].toarray())[0]
                if total_effects >= self.a:
                    self.State[i] = 1
            
            # recording after one-step iteration
            times += 1
            prop_path.append(self.State.mean())

            # print this step 
            if self.echo:
                print('第{}次迭代后:  {}的节点被激活'.format(times, prop_path[-1]))
            
            # stop the contagion if necessary
            if prop_path[-1] == prop_path[-2]:
                times = 9999
                csv_writer.writerow([self.mod, self.num_nodes, self.z, self.a, self.p_rewire, times])
                break

        else:
            # write csv
            csv_writer.writerow([self.mod, self.num_nodes, self.z, self.a, self.p_rewire, times])
            print('**模式{}:N={},z={}, a={}, p_rewire={}, 时长{}'.format(self.mod, self.num_nodes, self.z, self.a, self.p_rewire, times))

        f.close()

In [50]:
# 定义参数空间
p_rewire_vector = np.power(10, np.arange(-3, 0.1, .1)) # p_rewire   
a_vector =  [1, 2, 3]
mod_vector =  [1, 2, 3]
Repeat_time =  100 # 每个参数组合重复多少次

# 总次数
total_iters = p_rewire_vector.shape[0] * len(a_vector) * len(mod_vector) * Repeat_time 
# 计数器
count_iter = 0

# 循环
for mod in mod_vector:
    for a in a_vector:
        for p_rewire in p_rewire_vector:
            for i in range(Repeat_time):
                network_simulation = NS(num_nodes = 2000, 
                    p_rewire = p_rewire, 
                    z = 12, 
                    a = a, 
                    mod = mod, 
                    end_ratio = .99, 
                    echo = False)
                network_simulation.start_contagion()
                count_iter += 1
                print('{}/{}'.format(count_iter, total_iters))

**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长16
1/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长19
2/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长21
3/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长25
4/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长23
5/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长16
6/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长20
7/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长16
8/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长19
9/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长19
10/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长16
11/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长19
12/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长27
13/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长20
14/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长17
15/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长15
16/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长17
17/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.001, 时长18
18/27900
**模式1:N=2000,z=12, a=1, p_rewire=0.00